## LDA(Latent Dirichlet Allocation): scikit-learn

### 데이터 준비

In [1]:
import pandas as pd

In [2]:
news = pd.read_csv('./0306뉴스.csv') 

In [3]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_id        869 non-null    int64 
 1   title          869 non-null    object
 2   writer         869 non-null    object
 3   content        869 non-null    object
 4   writed_at      869 non-null    object
 5   url            869 non-null    object
 6   main_category  869 non-null    object
 7   sub_category   869 non-null    object
 8   platform       869 non-null    object
dtypes: int64(1), object(8)
memory usage: 61.2+ KB


In [4]:
news['sub_category'].unique()

array(['IT기기', 'Tech일반', '게임', '과학', '소프트웨어', '인터넷', '통신_모바일', '휴대폰통신'],
      dtype=object)

In [5]:
news.drop(news[news['sub_category'] == '과학'].index, inplace=True)

In [6]:
news['sub_category'].unique()

array(['IT기기', 'Tech일반', '게임', '소프트웨어', '인터넷', '통신_모바일', '휴대폰통신'],
      dtype=object)

In [7]:
news.dropna(subset=['content'], inplace=True) #content null 값 행 삭제 

In [8]:
news.drop_duplicates(subset=['content'], ignore_index=True, inplace=True) # 중복 기사 삭제 

In [9]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_id        761 non-null    int64 
 1   title          761 non-null    object
 2   writer         761 non-null    object
 3   content        761 non-null    object
 4   writed_at      761 non-null    object
 5   url            761 non-null    object
 6   main_category  761 non-null    object
 7   sub_category   761 non-null    object
 8   platform       761 non-null    object
dtypes: int64(1), object(8)
memory usage: 53.6+ KB


### 전처리 필요 부분 확인

In [ ]:
news['content'].iloc[0]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\n [서울=뉴시스] 정부가 올해 중 우리나라의 우주항공 분야 연구개발(R&D)을 전담할 '우주항공청'을 출범시킬 계획이다. 외부 전문가 채용 비중 제한을 없애고, 우수 인력을 유치하고자 주식백지신탁 예외까지 허용하는 등 우주항공청의 전문성을 높이기 위한 다양한 특례까지 마련된다. 과학기술정보통신부는 우주항공청 설립의 근거가 되는 '우주항공청의 설치 및 운영에 관한 특별법' 제정안을 2일부터 입법예고한다고 밝혔다. 특별법은 우주항공분야의 정책과 R&D, 산업육성 등을 총괄하는 중앙행정기관으로 우주항공청을 설치하고, 전문적이고 유연한 조직으로 운영하기 위한 원칙과 기능, 특례 등까지 명시하고 있다. 먼저 우주항공 정책 거버넌스를 구축하기 위해 과기정통부, 산업통상자원부 등의 부처가 개별적으로 수행하던 우주항공 관련 기술개발, 산업육성 지원, 인재양성 및 우주위험 대비 등의 기능을 모두 우주항공청으로 일원화했다. 이를 위해 기존 과기정통부 장관, 산지부 장관 등이 소관하던 우주항공 관련 법률인 '우주개발진흥법', '항공우주산업촉진법', '천문법' 등을 우주항공청장이 담당하도록 부칙을 통해 개정할 방침이다. 국가우주위원회 위원장은 국무총리에서 대통령으로 격상해 위상과 기능을 강화한다. 우주항공청장도 국가우주위에 새롭게 위원으로 추가해 실무위원회 위원장을 맡게 하는 등 우주항공청이 우주경제 시대를 여는 컨트롤 타워 역할을 하도록 했다. 우주항공청 조직의 경우 과기정통부 장관 소속으로 하되 전문성을 가진 조직으로 구성된다. 특히 우주항공청 조직 구성 원칙의 핵심은 유연하고 자율적인 운영이다. 이를 위해 우주항공청 내에 설치되는 별도의 본부가 우주항공 R&D와 산업 활성화를 전담하게 된다."

In [ ]:
# ▲ 없애기 
text = re.sub(r'[▲]','', text)

#뉴스기사 출처 ( = ), [ = ] 형태 없애기 코드 '\(.*=.*\) ' 
text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))

#[] 안에 들어가는 거 다 없애기? -> 그러면 안됨.. 

# 괄호 안 기자로 끝나는 거 지우기 
#re.sub(r'\[.*기자\]', '', text)

text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)

 # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)

In [ ]:
a = news['content'].str.contains('페이스북 /LeYN1 트위터 @yonhap_graphics')

print((news['content'].loc[a]))

7528     \n ,(서울=연합뉴스) 원형민 기자 = 정부가 미래산업의 성장동력을 확보하기 위한...
9133     \n ,(서울=연합뉴스) 원형민 기자 = 한국은행이 22일 발표한 '2월 기업경기실...
9937     \n ,(서울=연합뉴스) 원형민 기자 = 서방이 우크라이나 침공에 대한 제재로 러시...
16843    \n ,(서울=연합뉴스) 반종빈 기자 = 일본은 16일 한국에 대한 반도체 핵심 소...
Name: content, dtype: object


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
news['content'].iloc[9133]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\n ,(서울=연합뉴스) 원형민 기자 = 한국은행이 22일 발표한 '2월 기업경기실사지수(BSI)' 조사 결과에 따르면 이달 제조업 업황 BSI는 전월보다 3포인트 하락한 63을 기록했다., , 페이스북 /LeYN1 트위터 @yonhap_graphics,\n "

### tokenization

In [ ]:
!pip3 install mecab-python3

In [ ]:
!pip install konlpy

In [10]:
import konlpy
import re

# tokenization 함수를 만들어둠
def tokenize_korean_text(text):
    text = re.sub(r'[▲]','', text) 
    text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))
    text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)

    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return, 단어 간 관계 파악 위함으로 설정 안함 
    
    words = []
    stopwords = ['하는', '있다', '있는', '위해', '통해', '한다', '때문', '했다'] # 불용어 사전  
    for word, pos in Okt_morphs:
        if pos in ['Adjective', 'Verb', 'Noun', 'Alpha', 'Number'] and word not in stopwords:  # 이 경우에는 형용사, 동사, 명사, 영어, 숫자만 남김
            words.append(word)

    words_str = ' '.join(words)
    return words_str


# news['content']를 하나씩 tokenize해서 list로 저장
tokenized_list = []

for text in news['content']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

761
운전자 라면 당연히 교통법규 준수 생활화 해야 연속 된 벌점 면허 정지 이르는 처분 받기도 경찰청 운용 착한 운전 마일리지 를 평소 쌓아 두면 이 같은 경우 감경 혜택 받을 수 출처 바토 리먼 츠 1년 교통법규 무 위반 무사 달성 시 마일리지 10 점 적립 운전면허 보유 운전자 라면 누구 착한 운전 마일리지 적립 신청 할 수 교통법규 무 위반 무사 를 작성 후 1년 동안 서약 내용 지키면 마일리지 10 점 적립 할 수 있으며 횟수 제한 갱신 가능하다 경찰청 교통 민원 24 홈페이지 착한 운전 마일리지 신청 방법 출처 IT 동아 착한 운전 마일리지 적립 신청 하려면 면허증 챙겨 가까운 경찰서 교통 민원 실 방문 하면 된다 온라인 신청 원할 경우 경찰청 교통 민원 24 홈페이지 메인 화면 진입 하자 이후 상단 두 탭 운전면허 조사 예약 클릭 후 하위 카테고리 중 착한 운전 마일리지 신청 누르면 서약 작성 페이지 진입 할 수 착한 운전 마일리지 신청 페이지 출처 IT 동아 착한 운전 마일리지 서약 확인 출처 IT 동아 면허 정지 처분 시 착한 운전 마일리지 감경 혜택 서약 확인 를 발급 후 1년 과태료 범칙 금 아니라 사람 죽거나 다치게 교통사고 유발 하지 않으면 도로교통법 시행 규칙 별표 28 근거 해 착한 운전 마일리지 10 점 부여 된다 쌓은 착한 운전 마일리 혹시 운전자 벌점 40 점 이상 처분 면허 정지 대상 될 경우 사용 할 수 쌓은 마일리지 벌점 정지 일 사용자 중심 IT 널 IT 동아


In [11]:
tokenized_list[1]

'편집 자주 스마트폰 앱 모든 것 앱 으리 띵 사용자 중심 IT 널 IT 동아'

In [12]:
#단어가 1-20개만 포함된 corpus 삭제 
drop_corpus = []

for index in range(len(tokenized_list)):
    corpus = tokenized_list[index]
    if len(set(corpus.split())) < 21:   # 같은 단어 1-20개만 반복되는 corpus도 지우기 위해 set()을 사용
        news.drop(index=[index], axis=0, inplace=True)
        drop_corpus.append(corpus)
                
for corpus in drop_corpus:
    tokenized_list.remove(corpus)

news.reset_index(drop=True, inplace=True)

print(len(tokenized_list))
print(len(news))

730
730


### vectorization & LDA

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(tokenized_list)
word_count = X.toarray().sum(axis=1)
print("Total number of tokens:", word_count.sum())

Total number of tokens: 152367


In [15]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(tokenized_list)
unique_tokens = count_vectorizer.get_feature_names_out()
count_unique_tokens = len(unique_tokens)
print("Total number of unique tokens:", len(unique_tokens))

Total number of unique tokens: 15663


#### vectorization

In [16]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer_0 = CountVectorizer(max_df=0.8, max_features=round(count_unique_tokens*0.8), min_df=10, ngram_range=(1,2))
    # 10개의 문서 미만으로 등장하는 단어는 제외, 전체의 80% 이상으로 자주 등장하는 단어는 제외, 최대로 사용할 단어 수(백터 수)는 유니크한 토큰 수의 80% 
    # unigram, bigram 포함

feat_vect = count_vectorizer_0.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape) #문서의 수, 단어의 수(=유니크한 토큰 수)

CountVectorizer Shape: (730, 3100)


#### topic modeling: LDA

In [17]:
lda = LatentDirichletAllocation(n_components=7)  # 토픽 수는 7개로 설정
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=7)

#### 토픽별 연관어 출력

In [18]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names_out()를 통해 추출
feature_names = count_vectorizer_0.get_feature_names_out()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

Topic # 0
카카오 세븐 sm 헌혈 sk 나이 세븐 나이 진행 하이브 행사
Topic # 1
기술 기업 ai 사업 한국 서비스 개발 제공 투자 데이터
Topic # 2
kt 게임 대표 후보 선임 이사회 이사 체인 면접 블록
Topic # 3
전자 뉴스 스마트폰 아이 게임 아이 뉴스 삼성 올해 시장 지난해
Topic # 4
게임 거짓 유통 신세계 kt 지니 스튜디오 네오위즈 넷마블 아시아
Topic # 5
서비스 리뷰 고객 제공 기업 이용 관리 플랫폼 개인정보 클라우드
Topic # 6
게임 클레이튼 세포 개발 진행 기술 플랫폼 로봇 제공 카카오


### LDA 시각화

In [ ]:
!pip install pyLDAvis # pyLDAvis 라이브러리를 이용한 시각화 

In [ ]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer.get_feature_names_out())
pyLDAvis.display(vis) 

In [ ]:
pyLDAvis.show(vis) 

### 문서별 토픽 할당

#### 1. 각 문서별로 가장 가까운 topic으로 할당

In [19]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌 
doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df.head()

,Doc_Num,Topic,Percentage
0,0,1,0.518573
1,1,1,0.443633
2,2,3,0.757092
3,3,1,0.879446
4,4,1,0.666486


In [20]:
doc_topic_df = doc_topic_df.join(news)
doc_topic_df.head() #실제 뉴스기사와 join

,Doc_Num,Topic,Percentage,news_id,title,writer,content,writed_at,url,main_category,sub_category,platform
0,0,1,0.518573,595156,교통법규 준수로 쌓아가는 ’착한 운전 마일리지’,김동진,"운전자라면 당연히 교통법규 준수를 생활화해야 하지만, 때때로 연속된 벌점으로 면허...",2023-03-06 20:03:00,https://v.daum.net/v/20230306200300905,IT,IT기기,다음
1,1,1,0.443633,595158,'정수기만 렌털?'...매트리스 빌려 쓰세요,신영빈,가전 렌털업체들이 매트리스 시장 공략에 적극 나서고 있다. 그동안 정수기·공기청정기...,2023-03-06 17:35:00,https://v.daum.net/v/20230306173538566,IT,IT기기,다음
2,2,3,0.757092,595159,"엇갈린 1분기 실적…삼성 반도체에 '울고', LG 가전에 '웃나'",류은주,삼성전자와 LG전자가 1분기 실적에서 희비가 엇갈릴 것으로 관측된다. LG전자의 영...,2023-03-06 17:21:00,https://v.daum.net/v/20230306172126080,IT,IT기기,다음
3,3,1,0.879446,595160,"다이슨, 일상 속 문제 해결할 아이디어 찾는다",신영빈,글로벌 기술기업 다이슨이 일상 속 다양한 문제를 해결할 아이디어를 선보이는 국제 무...,2023-03-06 14:53:00,https://v.daum.net/v/20230306145326668,IT,IT기기,다음
4,4,1,0.666486,595161,"벨킨, 사운드폼 펄스 노캔 무선이어폰 출시",신영빈,벨킨이 사운드폼 펄스 노이즈 캔슬링 블루투스 무선 이어폰을 출시했다고 6일 밝혔다....,2023-03-06 14:45:00,https://v.daum.net/v/20230306144546341,IT,IT기기,다음


#### 2. 토픽별 문서 수 계산

In [21]:
doc_topic_df.groupby('Topic')[['Doc_Num']].count()

,Doc_Num
Topic,
0,66
1,298
2,70
3,110
4,42
5,73
6,71


#### 3. 토픽별로, 가장 높은 확률로 할당된 문서 top3 확인(이걸로 메인 뉴스 뽑기)

In [22]:
for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    print(top_pr_topics['content'].iloc[0])
    print(top_pr_topics['content'].iloc[1])
    print(top_pr_topics['content'].iloc[2], '\n')

Topic # 0 -----------------------------
기사내용 요약 원작 '세븐나이츠' 정식 계승…내년 상반기 CBT [서울=뉴시스]최은수 기자 = 넷마블은 모바일 역할수행게임(RPG) '세븐나이츠'를 정식 계승한 리메이크 작품 '더 세븐나이츠(The SevenKnights)'를 공개했다고 6일 밝혔다. '세븐나이츠'의 9주년을 기념해 공개된 더 세븐나이츠는 원작을 계승한 리메이크 프로젝트다. 이용자들이 가장 많이 즐겼던 시점의 재미를 다시 한번 선사하기 위해 원작의 장점은 계승, 발전시키고 부족한 부분은 최근 트렌드에 맞게 개선하고 보완할 계획이다 더 세븐나이츠는 기존의 스토리와 캐릭터성은 최대한 유지하는 동시에 새로운 재미를 제공하기 위해 준비 중이며, 내년 상반기 비공개베타테스트(CBT) 진행을 목표로 개발에 전념하고 있다. 이 프로젝트는 '세븐나이츠'부터 '세븐나이츠2'까지 오랜 기간 경험과 노하우를 쌓은 개발진이 이끌고 있으며, 구체적인 게임성 및 향후 일정에 대해서는 '세븐나이츠' 공식 커뮤니티를 통해 공지될 예정이다. 넷마블은 리메이크 프로젝트 공개 소식과 함께 더 세븐나이츠의 티저 영상을 공개했다. 원작의 사황 중 한명인 '에이스'가 검술을 연마하는 모습이 담긴 영상은 고퀄리티 카툰 스타일 그래픽으로 연출돼 기대감을 높이고 있다. 넷마블 관계자는 "오랜 기간 '세븐나이츠'를 사랑해주신 이용자분들에게 원작의 감동을 드리기 위해 리메이크 프로젝트 개발에 최선을 다하고 있다"며 "빠른 시일 내에 구체적인 게임성에 대해 안내해드릴 수 있도록 노력할 예정이며, 앞으로 많은 응원 부탁드린다"고 전했다. 한편, '세븐나이츠'는 지난 2014년 3월에 출시된 수집형 RPG로 화려한 그래픽과 연출, 500여종 이상의 캐릭터를 모으고 성장하는 재미를 갖춘 게임으로 국내뿐만 아니라 글로벌 시장에서 큰 성공을 기록한 바 있다. ☞공감언론 뉴시스 
넷마블의 RPG '세븐나이츠'가 9주년을 기념해 리메이크 버전 '더 세븐나이츠'로 돌아왔다. /사진=넷마

#### 키워드 요약, 문장 요약을 위해 토픽 7개만 들어있는 리스트 만들기

In [24]:
#content만 담겨있는 list
top7_topic_contents = []

for topic in range(len(doc_topic_df['Topic'].unique())):
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    content1 = top_pr_topics['content'].iloc[0]
    top7_topic_contents.append([content1])

print(top7_topic_contents) 

[['기사내용 요약 원작 \'세븐나이츠\' 정식 계승…내년 상반기 CBT [서울=뉴시스]최은수 기자 = 넷마블은 모바일 역할수행게임(RPG) \'세븐나이츠\'를 정식 계승한 리메이크 작품 \'더 세븐나이츠(The SevenKnights)\'를 공개했다고 6일 밝혔다. \'세븐나이츠\'의 9주년을 기념해 공개된 더 세븐나이츠는 원작을 계승한 리메이크 프로젝트다. 이용자들이 가장 많이 즐겼던 시점의 재미를 다시 한번 선사하기 위해 원작의 장점은 계승, 발전시키고 부족한 부분은 최근 트렌드에 맞게 개선하고 보완할 계획이다 더 세븐나이츠는 기존의 스토리와 캐릭터성은 최대한 유지하는 동시에 새로운 재미를 제공하기 위해 준비 중이며, 내년 상반기 비공개베타테스트(CBT) 진행을 목표로 개발에 전념하고 있다. 이 프로젝트는 \'세븐나이츠\'부터 \'세븐나이츠2\'까지 오랜 기간 경험과 노하우를 쌓은 개발진이 이끌고 있으며, 구체적인 게임성 및 향후 일정에 대해서는 \'세븐나이츠\' 공식 커뮤니티를 통해 공지될 예정이다. 넷마블은 리메이크 프로젝트 공개 소식과 함께 더 세븐나이츠의 티저 영상을 공개했다. 원작의 사황 중 한명인 \'에이스\'가 검술을 연마하는 모습이 담긴 영상은 고퀄리티 카툰 스타일 그래픽으로 연출돼 기대감을 높이고 있다. 넷마블 관계자는 "오랜 기간 \'세븐나이츠\'를 사랑해주신 이용자분들에게 원작의 감동을 드리기 위해 리메이크 프로젝트 개발에 최선을 다하고 있다"며 "빠른 시일 내에 구체적인 게임성에 대해 안내해드릴 수 있도록 노력할 예정이며, 앞으로 많은 응원 부탁드린다"고 전했다. 한편, \'세븐나이츠\'는 지난 2014년 3월에 출시된 수집형 RPG로 화려한 그래픽과 연출, 500여종 이상의 캐릭터를 모으고 성장하는 재미를 갖춘 게임으로 국내뿐만 아니라 글로벌 시장에서 큰 성공을 기록한 바 있다. ☞공감언론 뉴시스 '], ['연결기준 내부회계관리제도 확대 이슈가 시장 화두로 떠오르면서 대응을 위한 기업의 회계 프로세스 재정비 작업이 분주하다. 201

In [25]:
#news_id, title, content만 담겨있는 list
top7_topic = []

for topic in range(len(doc_topic_df['Topic'].unique())):
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    content1 = top_pr_topics.iloc[0]
    top7_topic.append([content1['news_id'], content1['title'], content1['content']])

print(top7_topic)

[[624571, "넷마블, 원작 리메이크 신작 '더 세븐나이츠' 공개", '기사내용 요약 원작 \'세븐나이츠\' 정식 계승…내년 상반기 CBT [서울=뉴시스]최은수 기자 = 넷마블은 모바일 역할수행게임(RPG) \'세븐나이츠\'를 정식 계승한 리메이크 작품 \'더 세븐나이츠(The SevenKnights)\'를 공개했다고 6일 밝혔다. \'세븐나이츠\'의 9주년을 기념해 공개된 더 세븐나이츠는 원작을 계승한 리메이크 프로젝트다. 이용자들이 가장 많이 즐겼던 시점의 재미를 다시 한번 선사하기 위해 원작의 장점은 계승, 발전시키고 부족한 부분은 최근 트렌드에 맞게 개선하고 보완할 계획이다 더 세븐나이츠는 기존의 스토리와 캐릭터성은 최대한 유지하는 동시에 새로운 재미를 제공하기 위해 준비 중이며, 내년 상반기 비공개베타테스트(CBT) 진행을 목표로 개발에 전념하고 있다. 이 프로젝트는 \'세븐나이츠\'부터 \'세븐나이츠2\'까지 오랜 기간 경험과 노하우를 쌓은 개발진이 이끌고 있으며, 구체적인 게임성 및 향후 일정에 대해서는 \'세븐나이츠\' 공식 커뮤니티를 통해 공지될 예정이다. 넷마블은 리메이크 프로젝트 공개 소식과 함께 더 세븐나이츠의 티저 영상을 공개했다. 원작의 사황 중 한명인 \'에이스\'가 검술을 연마하는 모습이 담긴 영상은 고퀄리티 카툰 스타일 그래픽으로 연출돼 기대감을 높이고 있다. 넷마블 관계자는 "오랜 기간 \'세븐나이츠\'를 사랑해주신 이용자분들에게 원작의 감동을 드리기 위해 리메이크 프로젝트 개발에 최선을 다하고 있다"며 "빠른 시일 내에 구체적인 게임성에 대해 안내해드릴 수 있도록 노력할 예정이며, 앞으로 많은 응원 부탁드린다"고 전했다. 한편, \'세븐나이츠\'는 지난 2014년 3월에 출시된 수집형 RPG로 화려한 그래픽과 연출, 500여종 이상의 캐릭터를 모으고 성장하는 재미를 갖춘 게임으로 국내뿐만 아니라 글로벌 시장에서 큰 성공을 기록한 바 있다. ☞공감언론 뉴시스 '], [617250, "[연결내부회계제도 대응]더존비즈온, 연결

In [26]:
top7_topic = pd.DataFrame(top7_topic, columns=['news_id', 'title', 'content'])


In [27]:
top7_topic

,news_id,title,content
0,624571,"넷마블, 원작 리메이크 신작 '더 세븐나이츠' 공개",기사내용 요약 원작 '세븐나이츠' 정식 계승…내년 상반기 CBT [서울=뉴시스]최은...
1,617250,"[연결내부회계제도 대응]더존비즈온, 연결결산시스템 'EFIS 10' 150곳 도입 달성",연결기준 내부회계관리제도 확대 이슈가 시장 화두로 떠오르면서 대응을 위한 기업의 회...
2,624577,"정치권 맹공에 혼돈의 KT, 차기 대표 후보자 7일 발표","-KT, 최종 후보 발표 ""예정대로 진행"" -사내외에선 후보 4인 일괄 사퇴설도 K..."
3,624550,"삼성, 신작 없이도 '인도 스마트폰 1위 탈환'…노태문, 약속지켰다",삼성전자 스마트폰이 인도에서 샤오미를 꺾고 점유율 1위에 올랐다. '가성비' 중심에...
4,605278,"신세계I&C, 네오위즈와 ‘맞손’...콘솔 패키지 게임 아시아 독점유통",신세계아이앤씨가 아시아 시장에서 독점 유통하는 네오위즈의 새 콘솔 플랫폼 게임 P의...
5,605277,“간편하고 생생한 사용후기 ‘U+콕’서 공유하세요”,U+콕 내 동영상 리뷰 서비스를 소개하는 직원들의 모습. [LG유플러스 제공] LG...
6,605201,클레이튼 “디플레이션 목표”…방향성은 명확하나 전술은 글쎄,서상민 클레이튼 재단 이사장이 6일 서울 강남구 EG빌딩에서 열린 기자 간담회에서 ...


# LDA: gensim

## tokenization

In [21]:
import konlpy
import re

def tokenize_korean_text(text):
    text = re.sub(r'[▲]','', text) 
    text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))
    text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)
    
    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)
    
    words = []
    stopwords = ['하는', '있다', '있는', '위해', '통해', '한다', '때문'] # 불용어 사전  
    for word, pos in Okt_morphs:
        if pos in ['Adjective', 'Verb', 'Noun', 'Alpha', 'Number'] and word not in stopwords:  # 이 경우에는 형용사, 동사, 명사, 영어, 숫자만 남김
            words.append(word)
    ## word를 이어붙인 string 형태가 아닌 word의 list를 return해주는 게 sklearn lda 준비 과정과의 차이
            
    return words


tokenized_list = []

for text in news['content']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

16214
['정부', '올해', '중', '우리나라', '우주', '항공', '분야', '연구개발', 'R', 'D', '전담', '할', '우주', '항공', '청', '출범', '시킬', '계획', '외부', '전문가', '채용', '비중', '제한', '없애고', '우수', '인력', '유치하고자', '주식', '신탁', '예외', '허용', '등', '우주', '항공', '청', '전문성', '높이기', '위', '다양한', '특례', '마련', '된다', '과학기술', '정보통신부', '우주', '항공', '청', '설립', '근거', '되는', '우주', '항공', '청', '설치', '및', '운영', '관', '특별법', '정안', '2일', '입법', '예고', '한다고', '밝혔다', '특별법', '우주', '항공', '분야', '정책', 'R', 'D', '산업', '육성', '등', '총괄', '중앙', '행정기관', '우주', '항공', '청', '설치', '전문', '유연한', '조직', '운영', '하기', '위', '원칙', '기능', '특례', '등', '명시', '먼저', '우주', '항공', '정책', '거버넌스', '구축', '하기', '과', '정통부', '산업', '통상', '자원부', '등', '부처', '개별', '수행', '하던', '우주', '항공', '관련', '기술', '개발', '산업', '육성', '지원', '인재', '양성', '및', '우주', '위험', '대비', '등', '기능', '모두', '우주', '항공', '청', '일원', '화했다', '이를', '기존', '과', '정통부', '장관', '산지', '부', '장관', '등', '소관', '하던', '우주', '항공', '관련', '법률', '우주개발', '진흥', '법', '항공우주', '산업', '촉진법', '문법', '등', '우주', '항공', '청장', '담당', '하도록', '부칙', '개정', '할', '방침', '국가', '우

In [23]:
drop_corpus = []

for index in range(len(tokenized_list)):
    corpus = tokenized_list[index]
    if len(set(corpus)) < 21:   # corpus 자체가 list 형태인게 sklearn lda 준비 과정과의 차이
        news.drop(index, axis='index', inplace=True)
        drop_corpus.append(corpus)
    
for corpus in drop_corpus:
    tokenized_list.remove(corpus)

news.reset_index(drop=True, inplace=True)

## vectorization & LDA

In [25]:
# bigram 생성에 필요한 library
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# vectorize & lda에 필요한 library
from gensim import corpora
from gensim.models.ldamodel import LdaModel

In [28]:
# Build the bigram models
bigram = Phrases(tokenized_list, min_count=10, threshold=20)
bigram_mod = Phraser(bigram)
 
# See example
print(bigram_mod[tokenized_list[0]])

['정부', '올해', '중', '우리나라', '우주_항공', '분야', '연구개발_R', 'D', '전담', '할', '우주_항공', '청', '출범', '시킬', '계획', '외부_전문가', '채용', '비중', '제한', '없애고', '우수_인력', '유치하고자', '주식_신탁', '예외_허용', '등', '우주_항공', '청', '전문성', '높이기_위', '다양한', '특례', '마련', '된다', '과학기술_정보통신부', '우주_항공', '청_설립', '근거', '되는', '우주_항공', '청_설치', '및', '운영', '관_특별법', '정안', '2일', '입법_예고', '한다고_밝혔다', '특별법_우주', '항공', '분야', '정책', 'R_D', '산업_육성', '등', '총괄', '중앙_행정기관', '우주_항공', '청_설치', '전문', '유연한_조직', '운영', '하기_위', '원칙', '기능', '특례', '등', '명시', '먼저', '우주_항공', '정책', '거버넌스', '구축', '하기', '과_정통부', '산업_통상', '자원부', '등', '부처', '개별', '수행', '하던', '우주_항공', '관련', '기술', '개발', '산업_육성', '지원', '인재_양성', '및', '우주', '위험', '대비', '등', '기능', '모두', '우주_항공', '청', '일원', '화했다', '이를', '기존', '과_정통부', '장관', '산지', '부_장관', '등', '소관', '하던', '우주_항공', '관련', '법률', '우주개발_진흥', '법', '항공우주_산업', '촉진법', '문법', '등', '우주_항공', '청장', '담당', '하도록', '부칙', '개정', '할', '방침', '국가', '우주', '위원회_위원장', '국무총리_대통령', '격상', '해', '위상', '기능', '강화한다', '우주_항공', '청장', '국가', '우주', '위', '새롭게', '위원', '추가', '해', '실무', 

In [29]:
# tokenized_list의 모든 문서에 대해 bigram을 생성해줌
words_bigram = [bigram_mod[doc] for doc in tokenized_list]

In [31]:
dictionary = corpora.Dictionary(words_bigram)  # 각 단어에 번호를 할당해줌
    # bigram 포함하는 과정을 생략하고 싶으면, 그냥 바로 여기에 tokenized_list를 넣어주면 됨

dictionary.filter_extremes(no_below=20, no_above=0.80)   
    # 20개의 문서 미만으로 등장하는 단어는 제외, 전체의 80% 이상으로 자주 등장하는 단어는 제외

corpus = [dictionary.doc2bow(text) for text in words_bigram]
print(corpus[0]) # 첫번째 corpus를 테스트로 출력: 몇 번째 단어가 몇 번 나왔는지 저장되어 있음

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 2), (16, 2), (17, 1), (18, 2), (19, 1), (20, 3), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 4), (68, 17), (69, 3), (70, 2), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 1), (93, 3), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 7), (100, 2), (101, 2), (102, 1), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 2)

In [33]:
ldamodel = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=20, iterations=500)  # 토픽 수: 6

ldamodel.print_topics(num_words=7)  # num_words=10이 default

[(0,
  '0.027*"기술" + 0.014*"기업" + 0.012*"산업" + 0.011*"개발" + 0.011*"지원" + 0.010*"사업" + 0.010*"해"'),
 (1,
  '0.034*"KT" + 0.012*"대표" + 0.009*"카카오" + 0.009*"경영" + 0.009*"는" + 0.008*"전" + 0.008*"기업"'),
 (2,
  '0.025*"서비스" + 0.015*"할_수" + 0.015*"고객" + 0.014*"데이터" + 0.011*"제공" + 0.010*"해" + 0.008*"관리"'),
 (3,
  '0.062*"AI" + 0.021*"챗_GPT" + 0.019*"기술" + 0.016*"를" + 0.015*"인공_지능" + 0.014*"는" + 0.012*"가"'),
 (4,
  '0.022*"것" + 0.011*"이" + 0.008*"그" + 0.007*"고" + 0.006*"미국" + 0.006*"수" + 0.006*"할"'),
 (5,
  '0.019*"서비스" + 0.016*"기업" + 0.016*"플랫폼" + 0.015*"대표" + 0.014*"사업" + 0.012*"글로벌" + 0.011*"콘텐츠"'),
 (6,
  '0.012*"것" + 0.008*"이" + 0.007*"해" + 0.006*"할_수" + 0.006*"입니다" + 0.005*"수" + 0.005*"된"'),
 (7,
  '0.023*"게임" + 0.010*"해" + 0.009*"된" + 0.008*"블록_체인" + 0.008*"것" + 0.007*"거래" + 0.007*"가상_자산"'),
 (8,
  '0.014*"시장" + 0.011*"지난해" + 0.010*"것" + 0.009*"투자" + 0.009*"매출" + 0.007*"증가" + 0.007*"미국"'),
 (9,
  '0.013*"제품" + 0.009*"전자" + 0.008*"를" + 0.008*"해" + 0.008*"할_수" + 0.008*"는" + 0.008*"의"')]

## LDA 시각화

In [36]:
import pyLDAvis.gensim  # gensim의 ldamodel에 최적화된 라이브러리

vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


## 문서별 토픽 할당

In [37]:
doc_per_topic_list = []

for n in range(len(corpus)):
    doc_topic = ldamodel[corpus[n]]
    doc_topic = sorted(doc_topic, key=lambda x: (x[1]), reverse=True)
    topic_most_pr = doc_topic[0][0]
    topic_pr = doc_topic[0][1]
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

# 실제 review 내용과 join
doc_topic_df = doc_topic_df.join(news)  

doc_topic_df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Doc_Num,Topic,Percentage,news_id,title,writer,content,writed_at,url,main_category,sub_category,platform
0,0,0,0.476281,346630,"정부, '우주항공청' 연내 개청…백지신탁 예외 등 특혜 쏟는다",NaN,\n [서울=뉴시스] 정부가 올해 중 우리나라의 우주항공 분야 연구개발(R&D)을 ...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
1,1,0,0.405812,346633,우주항공청 전문가 영입 `연봉 10억·외국인 허용`,이준기,\n '한국형 NASA(미 항공우주국)'로 출범하는 우주항공청이 최고의 민간 전문가...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
2,2,8,0.601316,347804,"박정호의 투자유치 '매직'…""SK쉴더스 IPO보다 더 이득""",선한결,"\n ""SK쉴더스의 기업공개(IPO)를 철회해서 아쉬웠는데, IPO보다 더 높은 가...",2023-03-01 18:03:05,NaN,IT/과학,IT 일반,네이버
3,3,3,0.481386,347806,글·그림·로고 제작·번역...인공지능에 다 맡겨 보니,NaN,\n 최근 텍스트 생성 인공지능(AI) 챗봇 챗GPT(ChatGPT)를 이용해 만든...,2023-03-01 18:02:03,NaN,IT/과학,IT 일반,네이버
4,4,8,0.587245,347807,"LG엔솔-혼다, 美 배터리 공장 '첫삽'…기공식 개최",이한얼,\n LG에너지솔루션은 28일(현지시간) 오하이오 주 파이에트 카운티 제퍼슨빌 인근...,2023-03-01 18:26:01,NaN,IT/과학,IT 일반,네이버
